In [ ]:
!pip install pyspark

     |████████████████████████████████| 212.4MB 83kB/s 
     |████████████████████████████████| 204kB 22.1MB/s 
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880768 sha256=9f6c60e37dd0ce88c1fbd88c06f615e56e6e718d7153c041ebcb74f19a7d0d48
  Stored in directory: /root/.cache/pip/wheels/40/1b/2c/30f43be2627857ab80062bef1527c0128f7b4070b6b2d02139
Successfully built pyspark


In [ ]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.classification import LogisticRegression


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('abc').getOrCreate()
# Chuẩn bị dữ liệu đào tạo từ danh sách các bộ giá trị (nhãn, tính năng).
training = spark.createDataFrame([
    (1.0, Vectors.dense([0.0, 1.1, 0.1])),
    (0.0, Vectors.dense([2.0, 1.0, -1.0])),
    (0.0, Vectors.dense([2.0, 1.3, 1.0])),
    (1.0, Vectors.dense([0.0, 1.2, -0.5]))], ["label", "features"])

In [ ]:
training.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0| [0.0,1.1,0.1]|
|  0.0|[2.0,1.0,-1.0]|
|  0.0| [2.0,1.3,1.0]|
|  1.0|[0.0,1.2,-0.5]|
+-----+--------------+



In [ ]:
# Tạo một thể hiện LogisticRegression. Ví dụ này là một Công cụ ước tính.
lr = LogisticRegression(maxIter=10, regParam=0.01)

In [ ]:
# In ra các thông số, tài liệu và bất kỳ giá trị mặc định nào
print("LogisticRegression parameters:\n" + lr.explainParams() + "\n")

LogisticRegression parameters:
aggregationDepth: suggested depth for treeAggregate (>= 2). (default: 2)
elasticNetParam: the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty. (default: 0.0)
family: The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial (default: auto)
featuresCol: features column name. (default: features)
fitIntercept: whether to fit an intercept term. (default: True)
labelCol: label column name. (default: label)
lowerBoundsOnCoefficients: The lower bounds on coefficients if fitting under bound constrained optimization. The bound matrix must be compatible with the shape (1, number of features) for binomial regression, or (number of classes, number of features) for multinomial regression. (undefined)
lowerBoundsOnIntercepts: The lower bounds on intercepts if fitting under bound constrained optimization. The bou

In [ ]:
# Learn a LogisticRegression model. This uses the parameters stored in lr.
model1 = lr.fit(training)

In [ ]:
# Vì model1 là Mô hình (tức là máy biến áp do Công cụ ước tính sản xuất),
# chúng ta có thể xem các tham số mà nó sử dụng trong fit ().
# Điều này in ra các cặp tham số (name: value), trong đó tên là ID duy nhất cho điều này
# Phiên bản LogisticRegression.
print("Model 1 was fit using parameters: ")
print(model1.extractParamMap())

Model 1 was fit using parameters: 
{Param(parent='LogisticRegression_8843034b63dd', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_8843034b63dd', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_8843034b63dd', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_8843034b63dd', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_8843034b63dd', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_8843034b63dd', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_8843034b63dd', name='maxBlockSizeInMB', do

In [ ]:
 #Theo cách khác, chúng tôi có thể chỉ định các tham số bằng cách sử dụng từ điển Python dưới dạng paramMap
paramMap = {lr.maxIter: 20}
paramMap[lr.maxIter] = 30  #Chỉ định 1 Tham số, ghi đè lên maxIter ban đầu.
#Chỉ định nhiều Tham số.
paramMap.update({lr.regParam: 0.1, lr.threshold: 0.55})  # type: ignore

In [ ]:
# Bạn có thể kết hợp paramMaps, là từ điển python.
# Thay đổi tên cột đầu ra
paramMap2 = {lr.probabilityCol: "myProbability"}  # type: ignore
paramMapCombined = paramMap.copy()
paramMapCombined.update(paramMap2)  # type: ignore

In [ ]:
# Bây giờ tìm hiểu một mô hình mới bằng cách sử dụng các tham số paramMapCombined.
# paramMapCombined ghi đè tất cả các tham số đã đặt trước đó thông qua các phương thức lr.set *
model2 = lr.fit(training, paramMapCombined)
print("Model 2 was fit using parameters: ")
print(model2.extractParamMap())

Model 2 was fit using parameters: 
{Param(parent='LogisticRegression_8843034b63dd', name='aggregationDepth', doc='suggested depth for treeAggregate (>= 2).'): 2, Param(parent='LogisticRegression_8843034b63dd', name='elasticNetParam', doc='the ElasticNet mixing parameter, in range [0, 1]. For alpha = 0, the penalty is an L2 penalty. For alpha = 1, it is an L1 penalty.'): 0.0, Param(parent='LogisticRegression_8843034b63dd', name='family', doc='The name of family which is a description of the label distribution to be used in the model. Supported options: auto, binomial, multinomial'): 'auto', Param(parent='LogisticRegression_8843034b63dd', name='featuresCol', doc='features column name.'): 'features', Param(parent='LogisticRegression_8843034b63dd', name='fitIntercept', doc='whether to fit an intercept term.'): True, Param(parent='LogisticRegression_8843034b63dd', name='labelCol', doc='label column name.'): 'label', Param(parent='LogisticRegression_8843034b63dd', name='maxBlockSizeInMB', do

In [ ]:
# Prepare test data
test = spark.createDataFrame([
    (1.0, Vectors.dense([-1.0, 1.5, 1.3])),
    (0.0, Vectors.dense([3.0, 2.0, -0.1])),
    (1.0, Vectors.dense([0.0, 2.2, -1.5]))], ["label", "features"])

In [ ]:
test.show()

+-----+--------------+
|label|      features|
+-----+--------------+
|  1.0|[-1.0,1.5,1.3]|
|  0.0|[3.0,2.0,-0.1]|
|  1.0|[0.0,2.2,-1.5]|
+-----+--------------+



In [ ]:
# Đưa ra dự đoán về dữ liệu thử nghiệm bằng phương thức Transformer.transform ().
# LogisticRegression.transform sẽ chỉ sử dụng cột 'tính năng'.
# Lưu ý rằng model2.transform () xuất ra cột "myProbability" thay vì cột thông thường
# cột 'xác suất' vì trước đây chúng tôi đã đổi tên thông số lr.probabilityCol.
prediction = model2.transform(test)
result = prediction.select("features", "label", "myProbability", "prediction") \
    .collect()

for row in result:
    print("features=%s, label=%s -> prob=%s, prediction=%s"
          % (row.features, row.label, row.myProbability, row.prediction))

features=[-1.0,1.5,1.3], label=1.0 -> prob=[0.05707304171034107,0.9429269582896589], prediction=1.0
features=[3.0,2.0,-0.1], label=0.0 -> prob=[0.9238522311704136,0.0761477688295864], prediction=0.0
features=[0.0,2.2,-1.5], label=1.0 -> prob=[0.10972776114780078,0.8902722388521992], prediction=1.0


----------------------------------------------------------------------------------------------------------------------------------------------

---



In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import HashingTF, Tokenizer

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

In [ ]:
# Chuẩn bị tài liệu đào tạo từ danh sách các bộ giá trị (id, văn bản, nhãn)
training = spark.createDataFrame([
    (0, "a b c d e spark", 1.0),
    (1, "b d", 0.0),
    (2, "spark f g h", 1.0),
    (3, "hadoop mapreduce", 0.0)
], ["id", "text", "label"])
training.show()

+---+----------------+-----+
| id|            text|label|
+---+----------------+-----+
|  0| a b c d e spark|  1.0|
|  1|             b d|  0.0|
|  2|     spark f g h|  1.0|
|  3|hadoop mapreduce|  0.0|
+---+----------------+-----+



In [ ]:
# Định cấu hình đường ống ML, bao gồm ba giai đoạn: tokenizer, hashingTF và lr
tokenizer = Tokenizer(inputCol="text", outputCol="words")
hashingTF = HashingTF(inputCol=tokenizer.getOutputCol(), outputCol="features")
lr = LogisticRegression(maxIter=10, regParam=0.001)
pipeline = Pipeline(stages=[tokenizer, hashingTF, lr])

In [ ]:
# Điều chỉnh đường ống cho các tài liệu đào tạo.
model = pipeline.fit(training)

In [ ]:
# Chuẩn bị các tài liệu thử nghiệm, là các bộ dữ liệu không được gắn nhãn (id, văn bản).
test = spark.createDataFrame([
    (4, "spark i j k"),
    (5, "l m n"),
    (6, "spark hadoop spark"),
    (7, "apache hadoop")
], ["id", "text"])
test.show()

+---+------------------+
| id|              text|
+---+------------------+
|  4|       spark i j k|
|  5|             l m n|
|  6|spark hadoop spark|
|  7|     apache hadoop|
+---+------------------+



In [ ]:
# Thực hiện dự đoán trên các tài liệu thử nghiệm và in các cột quan tâm.
prediction = model.transform(test)
selected = prediction.select("id", "text", "probability", "prediction")
for row in selected.collect():
    rid, text, prob, prediction = row  # type: ignore
    print(
        "(%d, %s) --> prob=%s, prediction=%f" % (
            rid, text, str(prob), prediction   # type: ignore
        )
    )

(4, spark i j k) --> prob=[0.1596407738787412,0.8403592261212588], prediction=1.000000
(5, l m n) --> prob=[0.8378325685476614,0.16216743145233858], prediction=0.000000
(6, spark hadoop spark) --> prob=[0.06926633132976266,0.9307336686702373], prediction=1.000000
(7, apache hadoop) --> prob=[0.9821575333444208,0.017842466655579203], prediction=0.000000


# **Phan Loai Hoi Quy**

**Hồi quy logistic nhị thức**

In [ ]:
from pyspark.ml.classification import LogisticRegression

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

# **Loại dữ liệu - API dựa trên RDD**

**Vectơ địa phương**

In [ ]:
import numpy as np
import scipy.sparse as sps
from pyspark.mllib.linalg import Vectors

In [ ]:
# Use a NumPy array as a dense vector.
dv1 = np.array([1.0, 0.0, 3.0])
# Use a Python list as a dense vector.
dv2 = [1.0, 0.0, 3.0]
# Create a SparseVector.
sv1 = Vectors.sparse(3, [0, 2], [1.0, 3.0])
# Use a single-column SciPy csc_matrix as a sparse vector.
sv2 = sps.csc_matrix((np.array([1.0, 3.0]), np.array([0, 2]), np.array([0, 2])), shape=(3, 1))

In [ ]:
print (dv1)
print (dv2)
print (sv1)
print (sv2)

[1. 0. 3.]
[1.0, 0.0, 3.0]
(3,[0,2],[1.0,3.0])
  (0, 0)	1.0
  (2, 0)	3.0


# **Điểm được gắn nhãn**

Một điểm được gắn nhãn là một vectơ cục bộ, dày đặc hoặc thưa thớt, được liên kết với một nhãn / phản hồi. Trong MLlib, các điểm được gắn nhãn được sử dụng trong các thuật toán học có giám sát. Chúng tôi sử dụng bộ đôi để lưu trữ một nhãn, vì vậy chúng tôi có thể sử dụng các điểm được gắn nhãn trong cả hồi quy và phân loại. Đối với phân loại nhị phân, nhãn phải là 0(âm) hoặc 1(dương). Đối với phân loại nhiều lớp, nhãn nên chỉ số lớp bắt đầu từ số không: 0, 1, 2, ....

In [ ]:
from pyspark.mllib.linalg import SparseVector
from pyspark.mllib.regression import LabeledPoint

# Create a labeled point with a positive label and a dense feature vector.
pos = LabeledPoint(1.0, [1.0, 0.0, 3.0])

# Create a labeled point with a negative label and a sparse feature vector.
neg = LabeledPoint(0.0, SparseVector(3, [0, 2], [1.0, 3.0]))

In [ ]:
print (pos)
print (neg)

(1.0,[1.0,0.0,3.0])
(0.0,(3,[0,2],[1.0,3.0]))


# **Ma trận cục bộ**

In [ ]:
from pyspark.mllib.linalg import Matrix, Matrices

# Create a dense matrix ((1.0, 2.0), (3.0, 4.0), (5.0, 6.0))
dm2 = Matrices.dense(3, 2, [1, 3, 5, 2, 4, 6])
print (dm2)

# Create a sparse matrix ((9.0, 0.0), (0.0, 8.0), (0.0, 6.0))
sm = Matrices.sparse(3, 2, [0, 1, 3], [0, 2, 1], [9, 6, 8])
print (sm)

DenseMatrix([[1., 2.],
             [3., 4.],
             [5., 6.]])
3 X 2 CSCMatrix
(0,0) 9.0
(2,1) 6.0
(1,1) 8.0


# **RowMatrix**

In [ ]:
from pyspark.mllib.linalg.distributed import RowMatrix
import pyspark
sc.stop()
# sc = pyspark.SparkContext("local", "My First Spark App")

sc = pyspark.SparkContext.getOrCreate()
# Create an RDD of vectors.
rows = sc.parallelize([[1, 2, 3], [4, 5, 6], [7, 8, 9], [10, 11, 12]])

print (row)
# Create a RowMatrix from an RDD of vectors.
mat = RowMatrix(rows)
print (mat)

# Get its size.
m = mat.numRows()  # 4
n = mat.numCols()  # 3

# Get the rows as an RDD of vectors again.
rowsRDD = mat.rows
print (rowsRDD)

Row(id=7, text='apache hadoop', probability=DenseVector([0.9822, 0.0178]), prediction=0.0)
MapPartitionsRDD[3] at mapPartitions at PythonMLLibAPI.scala:1342
